In [1]:
import numpy as np
import pandas as pd
import gc
import lightgbm as lgb
import warnings
import gc
from sklearn.preprocessing import StandardScaler
import matplotlib as  mpl
from matplotlib  import pyplot as plt
mpl.rcParams[u'font.sans-serif'] = ['simhei']
mpl.rcParams['axes.unicode_minus'] = False
from datetime import timedelta
from datetime import datetime
warnings.filterwarnings('ignore')

## 1.获取历史数据
针对特定场景数据做相应的处理，不具备通用性  

In [2]:
%%time
# 1. 读取数据
dataset = pd.read_csv('./train/dataset.csv', index_col=0)
dbd_faultcode = pd.read_csv('./train/dbd_faultcode.csv', encoding='GBK' )
train_data = dataset.merge(dbd_faultcode,how = 'left',on = ["SPN","FMI"])
up_data = train_data
up_data.COLLECTTIME = pd.to_datetime(up_data.COLLECTTIME)
up_data.sort_values(by='COLLECTTIME', inplace=True)
# 删除明显异常的行
drop_rows = [0, 1, 2, 3]
drop_rows.extend([x for x in range(1875076, 1875088)])
up_data.drop(drop_rows, axis=0, inplace=True)
# 默认填充为0
up_data["FLAG"] = 0
# 筛选出故障编码不为空的UNIQUENO(一个UNIQUENO可假设为一台机器)
prob_uniqs = up_data[up_data.CODE.notnull()].UNIQUENO.unique()

# 2.出现故障的时间段进行标记
'''
minutes = 300
'''
# 设定优先级，对于少数种类的故障，优先标记
code_prev = {
    1206: 0,
    1141: 1,
    1182: 2,
    1168: -1,
    1239: -1,
    1168: -1,
    1211: -1,
    1216: -1,
    1209: -1,
    1145: -1,
    1341: -1,
    1246: -1,
    1215: -1,
    1122: -1
}

delta = timedelta(minutes = 300)
left_delta = delta
right_delta = delta
# 从有故障的设备列表中取出一台设备
for dev_id in prob_uniqs:
    # 取出这台设备的全部数据
    cdf = up_data[up_data.UNIQUENO == dev_id].copy()
    cdf['index'] = cdf.index
    cdf.reset_index(inplace=True)
    # 取出这台设备故障码不为空的数据
    notna_cdf = cdf[cdf.CODE.notna()]
#     print(datetime.now(), dev_id, cdf.shape[0], notna_cdf.shape[0])
    cdf_len = cdf.shape[0]
    # 在这台设备数据中遍历不为空的位置
    for i in notna_cdf.index:
        dt = cdf.at[i, 'COLLECTTIME']
        code = cdf.at[i, 'CODE']
        # 在前后时间间隔内打标签
        for y in range(i, -1, -1):
            y_code = up_data.at[cdf.at[y, 'index'], 'FLAG']
            if y_code > 0 and code_prev[y_code] >= code_prev[code]:
                break
            elif cdf.at[y, 'COLLECTTIME'] >= dt - left_delta:
                up_data.at[cdf.at[y, 'index'], 'FLAG'] = code
            else:
                break
        
        for y in range(i + 1, cdf_len, 1):
            y_code = up_data.at[cdf.at[y, 'index'], 'FLAG']
            if y_code > 0 and code_prev[y_code] >= code_prev[code]:
                break
            elif cdf.at[y, 'COLLECTTIME'] <= dt + right_delta:
                up_data.at[cdf.at[y, 'index'], 'FLAG'] = code
            else:
                break

# 3.保留有用的列
#columns_filter,util_cols由人工提供
columns_filter = ['CD_VP_UNIQUENO', 'AREA', 'ROAD', 'LONGITUDE', 'LATITUDE', 'SPEED',
       'DIRECTION', 'ALTITUDE', 'ACCSTATE', 'POWERDOWNSTATE', 'LOCATIONSTATE',
       'WORKHOURS', 'ACCRUNTIMES', 'ACCWORKTIMES', 'PRESSURE',
       'WATERTEMPERATURE', 'OILLOCATION', 'OILTEMPERATURE', 'OILPRESSURE',
       'ROUNDSPEED', 'ACCEXCCOUNT', 'GSMSIGNAL', 'WORKDETAILID',
       'VEHICLESTATUS', 'TERMINALSTATUS', 'SATELLITE', 'TERMINALEL_OUTSIDE',
       'TERMINALEL_INSIDE', 'INSIDE_TEMPERATURE', 'INSIDE_HUMIDITY',
       'MILENUMBER', 'OILVALUE1', 'OILVALUE2', 'MIXSTATE', 'QUSHOILTIMES',
       'ENVTEMPERATURE', 'PULSESIGNS', 'ENVHUMIDITY', 'OILPERCENT1',
       'ELCANDVOL', 'WIRELESSTEMP1', 'WIRELESSTEMP2', 'WIRELESSTEMP3',
       'WIRELESSTEMP4', 'SIMNO', 'OPTOILLEVEL', 'ENGINELOADSTATE', 'CMDTYPE',
       'ISANALYSED', 'VERSIONNUM', 'LOCK_INSTRUMENT', 'ALARM_0010',
       'ALARM_0003', 'ALARM_0008', 'ALARM_0005', 'ALARM_0006', 'ALARM_0007',
       'ALARM_0009', 'LOWVOLTAGESTATUS', 'ALARM_0002', 'ALARM_0004',
       'ALARM_0001', 'THROTTLECURRENT', 'CURRENTSPEEDLOADPERCENTAGE',
       'ENGINETORQUEPERCENT', 'ROTATIONALSPEED', 'ENGINERUNTIMES',
       'COOLANTTEMPERATURE', 'ENGINEOILTEMPERATURE', 'INTAKETEMPERATURE',
       'TEMPERATURE', 'LUBRICATINGOILPRESSURE', 'COOLANTLEVEL',
       'INTAKEMANIFOLDTEMPERATURE', 'FUELUSETOTAL', 'OILCONSUMPTIONRATE',
       'CURRENTFAULTCOUNT1', 'CURRENTFAULTCOUNT2', 'CURRENTFAULTCOUNT3',
       'CURRENTFAULTCOUNT4', 'CURRENTFAULTCOUNT5', 'CURRENTFAULTCOUNT6','FLAG','UNIQUENO','COLLECTTIME']
util_cols = ['COLLECTTIME','FLAG','UNIQUENO']

# 筛选出这四类故障的数据
x_train = up_data[up_data.FLAG.isin({0,1206,1141,1182})]
x_data = x_train[columns_filter]
y_train = x_data['FLAG']
y_train.replace(1206,1,inplace=True)
y_train.replace(1141,2,inplace=True)
y_train.replace(1182,3,inplace=True)
x_data['FLAG'] = y_train.values 
x_data.index = pd.to_datetime(x_data['COLLECTTIME'])

train_x_cols = ['UNIQUENO','CD_VP_UNIQUENO', 'AREA', 'ROAD', 'LONGITUDE', 'LATITUDE', 'SPEED',
       'DIRECTION', 'ALTITUDE', 'ACCSTATE', 'POWERDOWNSTATE', 'LOCATIONSTATE',
       'WORKHOURS', 'ACCRUNTIMES', 'ACCWORKTIMES', 'PRESSURE',
       'WATERTEMPERATURE', 'OILLOCATION', 'OILTEMPERATURE', 'OILPRESSURE',
       'ROUNDSPEED', 'ACCEXCCOUNT', 'GSMSIGNAL', 'WORKDETAILID',
       'VEHICLESTATUS', 'TERMINALSTATUS', 'SATELLITE', 'TERMINALEL_OUTSIDE',
       'TERMINALEL_INSIDE', 'INSIDE_TEMPERATURE', 'INSIDE_HUMIDITY',
       'MILENUMBER', 'OILVALUE1', 'OILVALUE2', 'MIXSTATE', 'QUSHOILTIMES',
       'ENVTEMPERATURE', 'PULSESIGNS', 'ENVHUMIDITY', 'OILPERCENT1',
       'ELCANDVOL', 'WIRELESSTEMP1', 'WIRELESSTEMP2', 'WIRELESSTEMP3',
       'WIRELESSTEMP4', 'SIMNO', 'OPTOILLEVEL', 'ENGINELOADSTATE', 'CMDTYPE',
       'ISANALYSED', 'VERSIONNUM', 'LOCK_INSTRUMENT', 'ALARM_0010',
       'ALARM_0003', 'ALARM_0008', 'ALARM_0005', 'ALARM_0006', 'ALARM_0007',
       'ALARM_0009', 'LOWVOLTAGESTATUS', 'ALARM_0002', 'ALARM_0004',
       'ALARM_0001', 'THROTTLECURRENT', 'CURRENTSPEEDLOADPERCENTAGE',
       'ENGINETORQUEPERCENT', 'ROTATIONALSPEED', 'ENGINERUNTIMES',
       'COOLANTTEMPERATURE', 'ENGINEOILTEMPERATURE', 'INTAKETEMPERATURE',
       'TEMPERATURE', 'LUBRICATINGOILPRESSURE', 'COOLANTLEVEL',
       'INTAKEMANIFOLDTEMPERATURE', 'FUELUSETOTAL', 'OILCONSUMPTIONRATE',
       'CURRENTFAULTCOUNT1', 'CURRENTFAULTCOUNT2', 'CURRENTFAULTCOUNT3',
       'CURRENTFAULTCOUNT4', 'CURRENTFAULTCOUNT5', 'CURRENTFAULTCOUNT6']
train_y_col = ['UNIQUENO','FLAG']
his_x = x_data[train_x_cols]
his_y = x_data[train_y_col]

MemoryError: 

his_x,his_y为历史数据，用于接下来的模型训练

## 2.数据处理

In [3]:
# 0.自定义历史数据的时间范围
start = '2017-01-01'
end = '2018-12-12'
def get_train(his_x,his_y,start=start, end=end):
    return his_x[start:end],his_y[start:end]
train_ax, train_ay = get_train(his_x,his_y,start='2017-01-01', end='2018-12-12')

# 1.数据预处理
# <1>删除dup_col
train_x = train_ax.drop(columns=['UNIQUENO']).copy()
train_y = train_ay.drop(columns=['UNIQUENO']).copy()
null_filter = [col for col in train_x.columns if not train_x[col].isnull().all()] 
train_x = train_x[null_filter] # 不含UNIQUENO
train_UNIQUENO = train_ax[['UNIQUENO']] 
                  
# <2>缺失值处理
train_x.fillna(0,inplace=True)
train_xy = pd.concat([train_x,train_y], axis=1)
# <3>取FLAG=0的数据，即健康数据
train_x_baseline = train_xy[train_xy.FLAG==0].drop(columns=['FLAG'])
train_x_degraded = train_xy[train_xy.FLAG!=0].drop(columns=['FLAG'])


NameError: name 'his_x' is not defined

In [ ]:
# 3.计算几种metrics
# <1>fisher metric
cols = train_x_baseline.columns
m1 = train_x_baseline.mean().values
m2 = train_x_degraded.mean().values
var1 = train_x_baseline.var().values
var2 = train_x_degraded.var().values
a = (m1-m2)*(m1-m2)
b1 = var1
b2 = var2
b = b1+b2
fisher_score = a/b
fisher_rank = pd.Series(data=fisher_score,index=cols).rank(na_option='drop',ascending=False).sort_values() # rank()不会改变顺序

# <2> monotonicity
# train_x_smooth = train_xy.drop(columns=['FLAG']).rolling(window = 5, min_periods=1, center = False).mean()

# 局部加权线性回归
# def lwlr(x, X, Y, k):
#     ''' 局部加权线性回归，给定一个点，获取相应权重矩阵并返回回归系数
#     '''
#     m = X.shape[0]
#     print(m)
#     # 创建针对x的权重矩阵
#     W = np.matrix(np.zeros((m, m)))
#     for i in range(m):
#         xi = np.array(X[i][0])
#         x = np.array(x)
#         W[i, i] = np.exp((np.linalg.norm(x - xi))/(-2*k**2))
#     # 获取此点相应的回归系数
#     xWx = X.T*W*X
#     if np.linalg.det(xWx) == 0:
#         print('xWx is a singular matrix')
#         return
#     w = xWx.I*X.T*W*Y
#     return w

# lwlr(train_x_smooth.iloc[0,0], np.arange(len(train_x_smooth)), train_x_smooth.iloc[:,0].values.reshape(-1,1), 0.3)

In [ ]:
# 4.根据metric来排序特征，然后选择特征
feature_rank = fisher_rank
feature_rank = feature_rank
selcted_features = list(feature_rank.index[:int(0.4*len(feature_rank))])
train_x_baseline_sel = train_x_baseline[selcted_features]
train_x_degraded_sel = train_x_degraded[selcted_features]
train_x_sel = train_x[selcted_features]


# 5.数据标准化
# <1>定义baseline数据标准化函数
def standard_scale(x):
    if x.shape[0]==0:
        return x,0,0
    idx = x.index
    m = x.mean()
    std = x.std()
    scaler = StandardScaler()   
    x_arr = scaler.fit_transform(x)
    return pd.DataFrame(x_arr, index = idx, columns =x.columns),m,std

# <2>使用健康数据的m_b和std_b，定义对所有测试的数据做归一化标准化的函数
def standard_scale_new(data,m_b,std_b):
    cols = std_b[std_b==0].index
    data_s = (data-m_b)/std_b
    data_s[cols] = 0
    return data_s

train_x_baseline_stand,m_b,std_b = standard_scale(train_x_baseline_sel)

train_x_baseline_stand(DataFrame): 标准化的健康的特征数据   
selcted_features(list):筛选出来的特征   
train_x(DataFrame): 所有特征数据
train_y(DataFrame): 所有的标签(FLAG)
train_UNIQUENO(DataFrame):UNIQUENO  

## 3.健康模型

In [ ]:
# pca and T2
import joblib
import os
from sklearn.decomposition import PCA
def dimension_reduction(x,whiten=True):
    pca = PCA(n_components=0.95,whiten=whiten).fit(x)
    return pca

def cal_T2(data,pca):
    T2=[]
    projected_data = pca.transform(data)
    eigenvalue_matrix = np.mat(np.diag(pca.singular_values_ ))
    for x in projected_data:
        tmp = np.mat(x)*np.linalg.inv(eigenvalue_matrix)*np.mat(x).T
        T2.append(tmp[0,0])
    return T2
      
# 使用归一化的健康数据
pca = dimension_reduction(train_x_baseline_stand.values,whiten=False)
if not os.path.exists('model'):
    os.mkdir('./model')
# 保存pca模型
joblib.dump(pca,'./model/pca_%s_%s.pkl'%(start,end))

健康模型pca

## 4.概览

In [110]:
# train_ax.UNIQUENO.value_counts()[80:100]

In [ ]:
# 给定UNIQUENO，计算time-T2
'''
201706130830038670
201707201730009538
201801261730005948
201705272357138269    
'''

'''
全为健康的样本：
201706130830038669
201601250830021604
201612151430019861
201702251130053444
'''
UNIQUENO = 201707201730009538                                 
train_unino = train_x_sel[train_UNIQUENO.UNIQUENO==UNIQUENO]
train_unino_stand = standard_scale_new(train_unino,m_b,std_b)

T2 = cal_T2(train_unino_stand,pca)
T2 = pd.DataFrame(data=T2,columns=['T2'], index=train_unino.index)
# T2_normal = T2/T2.max() # 归一化
# T2_normal.plot()
T2.plot()
(T2.ewm(alpha=0.4).mean()).plot()
train_ay[train_ay.UNIQUENO==UNIQUENO].drop(columns=['UNIQUENO']).plot()

In [121]:
train_degraded_stand = standard_scale_new(train_x_degraded_sel,m_b,std_b)
T2_degraded = cal_T2(train_degraded_stand,pca)
T2_degraded = pd.DataFrame(data=T2_degraded,columns=['T2_degraded'], index=train_degraded_stand.index)

## 4.指定时间间隔，UNIQUENO，返回 可以直接计算数据（已做过相同的特征处理）

In [ ]:
import os
import datetime as dt
from datetime import timedelta


def gen_interval(data,y,m_b,std_b,UNIQUENO,selcted_features,start='2017-03-01',end='2018-10-12'):

    # 1.选数据
    # <1> 选设备
    data_unino = data[data.UNIQUENO==UNIQUENO].drop(columns=['UNIQUENO'])
    y_unino = y[y.UNIQUENO==UNIQUENO].drop(columns=['UNIQUENO'])

    # <2> 选时间片段
    data_date = data_unino[start:end]
    y_date = y_unino[start:end]
    data_date.sort_index(inplace=True)
    y_date.sort_index(inplace=True)
    # <3> 选特征
    data_sel = data_date[selcted_features]
    
    # 3.相同的特征处理
    # <1> fillna
    data_sel.fillna(0,inplace=True)
    # <2>标准化
    data_stand = standard_scale_new(data_sel,m_b,std_b)
    return data_stand,y_date
    

In [4]:
# data_stand,y_date = gen_interval(his_x,his_y,m_b,std_b,UNIQUENO,selcted_features,start='2017-03-01',end='2018-10-12')

## 4.健康分排序

In [308]:
# def cal_all_T2(x,y,m_b,std_b,selcted_features,start='2017-03-01',end='2018-10-12'):
#     unino = data.UNIQUENO.value_counts().index[:100]
#     res = dict()
#     for UNIQUENO in unino:
#         # 1.提取符合条件的数据
#         data = gen_interval(x, y, m_b, std_b, UNIQUENO, selcted_features, start, end)
#         # 2.计算T2值
#         if data.empty:
#             print('%s至%s,id为%d的设备无数据！'%(start,end,UNIQUENO))
#             return None
#         T2 = cal_T2(data_sel,pca)
#         T2 = pd.DataFrame(data=T2,columns=['T2'], index=data.index)
#         T2_ewm = T2.ewm(alpha=0.4).mean()
#         # 输出T2_ewm(DataFrame)
#     return T2_ewm

def cal_all_T2(x,y,m_b,std_b,selcted_features,start='2017-03-01',end='2018-10-12'):
    unino = y.UNIQUENO.value_counts().index[:100]
    res = dict()
    for UNIQUENO in unino:
        # 1.提取符合条件的数据
        data,_ = gen_interval(x, y, m_b, std_b, UNIQUENO, selcted_features, start, end)
        # 2.计算T2值
        if data.empty:
            print('%s至%s,id为%d的设备无数据！'%(start,end,UNIQUENO))
            continue
        T2 = cal_T2(data,pca)
        T2 = pd.DataFrame(data=T2,columns=['T2'], index=data.index)
        T2_ewm = T2.ewm(alpha=0.4).mean()
        T2_ewm_mean = T2_ewm['T2'].mean()
        res[UNIQUENO] = T2_ewm_mean
        # 输出T2_ewm(DataFrame)
    score_rank = pd.Series(res).rank(method='dense',ascending=True).sort_values() # T2值大，排名靠后
    return score_rank

In [309]:
%%time
# 计算指定天的分数
score_rank = cal_all_T2(his_x,his_y,m_b,std_b,selcted_features,start='2017-03-01',end='2018-10-12')

CPU times: user 1min 8s, sys: 6min 6s, total: 7min 14s
Wall time: 36.6 s


# 5.画指定时间间隔，UNIQUENO

In [1]:
def plot_data(his_x,his_y,m_b,std_b,UNIQUENO,selcted_features,start='2017-03-01',end='2018-10-12'):
    data_stand,y_date = gen_interval(his_x,his_y,m_b,std_b,UNIQUENO,selcted_features,start='2017-03-01',end='2018-10-12')
    T2 = cal_T2(data,pca)
    T2 = pd.DataFrame(data=T2,columns=['T2'], index=data.index)
    T2_ewm = T2.ewm(alpha=0.4).mean()
    T2_ewm.plot()
    y_date.plot()
    return T2_ewm,y_date

In [2]:
T2_ewm,y_date = plot_data(his_x,his_y,m_b,std_b,UNIQUENO,selcted_features,start='2017-03-01',end='2018-10-12')

NameError: name 'his_x' is not defined

# 5.计算每一维的贡献度  
需要归一化后的数据

In [238]:
def cal_sep_T2(data,pca):
    '''
    data(DataFrame):m个样本，n个维度
    '''
    # 计算总得分
#     T2 = cal_T2(data,pca)
    # 计算每个维度的得分
    T2_cols = []
    cols = list(data.columns)
    for i in range(len(cols)):
        tmp = np.zeros(data.shape)
        tmp[:,i] = data.iloc[:,i].values
        T2_col = cal_T2(tmp,pca)
        T2_cols.append(T2_col)
        
#     T2 =np.array(T2).reshape((-1,1))
    T2_cols = np.array(T2_cols).T
#     T2_all = np.concatenate((T2,T2_cols),axis=1)
#     return pd.DataFrame(data=T2_all, columns=['T2']+cols, index = data.index)
    return pd.DataFrame(data=T2_cols,columns=cols,index=data.index)

def cal_contrib(data,pca):
    result = cal_sep_T2(data,pca)
    res_mean = result.mean(axis=0)
    res_sum = res_mean.sum()
    contrib = res_mean/res_sum
    contrib = contrib.sort_values(ascending=False)
    return contrib

In [262]:
contrib.mean()

0.049999999999999996

In [228]:
# for test

# x = np.arange(20).reshape(-1,20)
# cal_T2(x,pca)



# y = np.diag(np.arange(20))
# sum(cal_T2(y,pca))

# x_t = pca.transform(x)

# a = pca.components_

# x.shape

# x_t

# cal_T2(x,pca)

# y.shape

# y_t = pca.transform(y)

# y_t.sum(axis=0)

# sum(cal_T2(y,pca))